In [18]:
from scipy import io
import numpy as np
import os
import tensorflow as tf
import tensorflow.keras.backend as K
import math

from TA_CSPNN import *

In [2]:
for file in os.listdir('matlab_data/'):
    if not file == "A01T.mat":
        continue
    print(file)
    
    data = io.matlab.loadmat("matlab_data/" + file)['data']
    labels = io.matlab.loadmat('true_labels/' + file)['classlabel']
    break
print(data.shape)
print(labels.shape)

A01T.mat
(22, 250, 288)
(288, 1)


In [3]:
num_samples = data.shape[2]

train_size = int(num_samples * .9) # 90/10 train validation split
val_size = num_samples - train_size
print(f"num train samples: {train_size}")
print(f"num val samples: {val_size}")


num train samples: 259
num val samples: 29


In [4]:
rand_idx = np.random.randint(0,288, num_samples)
train_idx = rand_idx[:train_size]
val_idx = rand_idx[train_size:]

x_train = data[:,:,train_idx]
y_train = labels[train_idx]

x_val = data[:,:,val_idx]
y_val = labels[val_idx]
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape}")
print(f"y_val shape: {y_val.shape}")

x_train shape: (22, 250, 259)
y_train shape: (259, 1)
x_val shape: (22, 250, 29)
y_val shape: (29, 1)


In [5]:
x_train, y_train = K.constant(x_train), K.constant(y_train)
x_val, y_val = K.constant(x_val), K.constant(y_val)

In [6]:
x_train = tf.reshape(x_train, [259, 1, 22, 250])
x_val = tf.reshape(x_val, [29, 1, 22, 250])

In [7]:
print(f"x_train shape: {x_train.shape}")
print(f"x_val shape: {x_val.shape}")

x_train shape: (259, 1, 22, 250)
x_val shape: (29, 1, 22, 250)


In [8]:
class model_config(object):
    def __init__(self):
        self.channels = 22
        self.timesamples = 250
        self.timeKernelLen = 64
        self.num_classes = 4
        self.Ft = 8
        self.Fs = 2

In [9]:
config = model_config()
model = TA_CSPNN(config.num_classes, Channels=config.channels, Timesamples=config.timesamples,
                timeKernelLen = config.timeKernelLen, Ft=config.Ft, Fs=config.Fs)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

In [11]:
model.compile(
    optimizer = Adam(),
    loss = CategoricalCrossentropy(from_logits=True)
)

In [12]:
# input shape is (trials, 1, channels, time_samples)

In [19]:
BATCH_SIZE=32
compute_steps_per_epoch = lambda x: int(math.ceil(1. * x / BATCH_SIZE))
steps_per_epoch = compute_steps_per_epoch(train_size)
val_steps = compute_steps_per_epoch(val_size)

In [21]:
history = model.fit(x_train, y_train, epochs=5, validation_data=((x_val, y_val)), steps_per_epoch=steps_per_epoch,
                   validation_steps=val_steps)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/5


InvalidArgumentError: Default AvgPoolingOp only supports NHWC on device type CPU
	 [[{{node average_pooling2d/AvgPool}}]]